In [1]:
from pymongo import MongoClient
from pprint import pprint
import ast #for converting string into dict 
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import ChatPromptTemplate
from typing import List, Dict, Any


In [2]:

# Simulated parsed user query
parsed_info = {
    "food_item": ["pizza"],
    "preferences": ["wine"],
    "aspects": ["staff"]
}

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")
collection = client["Reviews"]["Barcelona"]




In [3]:
def build_mongo_query(parsed_info: Dict[str, List[str]]) -> Dict[str, Any]:
      
  # Merge all values from food_item, preferences, and aspects
  search_aspects = []
  for key in ["food_item", "preferences", "aspects"]:
    search_aspects.extend(parsed_info.get(key, []))
    
  # If nothing to search, return empty query (match all)
  if not search_aspects:
    return "⚠️ Be more specif, no aspect was found"
  
  query = {}
  # Create one regex matcher per term (case-insensitive, substring match)
  regex_filters = [
    {"aspect_keys": {"$elemMatch": {"$regex": term, "$options": "i"}}}
    for term in search_aspects] 

  query = {"$or": regex_filters}
  return query  


In [4]:

# Build the query
mongo_query = build_mongo_query(parsed_info)

# Run it
results = list(collection.find(mongo_query).limit(10))

# Output
print(f"✅ Found {collection.count_documents(mongo_query)} matching documents.")
print("🔍 Showing up to 5 sample results:\n")
for doc in results:
    print(f"🍽️ Restaurant: {doc.get('restaurant_name')}")
    print(f"📝 Aspects: {doc.get('aspect_keys')}")
    print()


✅ Found 23 matching documents.
🔍 Showing up to 5 sample results:

🍽️ Restaurant: Sports_Bar_Italian_Food_C_Ample
📝 Aspects: ['service', 'quality of the pizza', 'staff']

🍽️ Restaurant: Sports_Bar_Italian_Food_C_Ample
📝 Aspects: ['pizzas', 'foccacia', 'vegetable and meat lasagne']

🍽️ Restaurant: Sports_Bar_Italian_Food_C_Ample
📝 Aspects: ['napolitanian pizza']

🍽️ Restaurant: Sports_Bar_Italian_Food_C_Ample
📝 Aspects: ['service', 'quality of the ingredients', 'Red wines']

🍽️ Restaurant: Bodega_Biarritz_1881
📝 Aspects: ['server', 'Kate', 'wine list']

🍽️ Restaurant: Bodega_Biarritz_1881
📝 Aspects: ['food', 'atmospher', 'staff']

🍽️ Restaurant: Bodega_Biarritz_1881
📝 Aspects: ['food', 'wine', 'priced']

🍽️ Restaurant: Bodega_Biarritz_1881
📝 Aspects: ['selection', 'staff', 'food']

🍽️ Restaurant: Bodega_Biarritz_1881
📝 Aspects: ['food', 'staff', 'service', 'value']

🍽️ Restaurant: Con_Gracia
📝 Aspects: ['food', 'wine paring']



In [5]:
import sys
import os

# Add the parent folder "consigliami" to Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Now import the class from scripts/DB_querying.py
from scripts.DB_querying import RestaurantRecommender


In [6]:
pizza_docs = list(collection.find(
    {"aspect_keys": {"$elemMatch": {"$regex": "pizza", "$options": "i"}}}
).limit(5))

for doc in pizza_docs:
    print(doc["restaurant_name"], doc.get("aspect_keys"))


Sports_Bar_Italian_Food_C_Ample ['service', 'quality of the pizza', 'staff']
Sports_Bar_Italian_Food_C_Ample ['pizzas', 'foccacia', 'vegetable and meat lasagne']
Sports_Bar_Italian_Food_C_Ample ['napolitanian pizza']
We_love_italy_Pasta_Pizza_Restaurante_Barcelona ['pizza', 'pasta', 'food', 'service']
We_love_italy_Pasta_Pizza_Restaurante_Barcelona ['pizza', 'crust', 'staff', 'prices', 'tables outside']


In [ ]:
from pprint import pprint
from pymongo import MongoClient

# ========== Setup Parameters ==========
DB_NAME = "Reviews"
COLLECTION_NAME = "Barcelona"
MODEL_NAME = "llama3:8b"

# ========== Instantiate the Class ==========
recommender = RestaurantRecommender(DB_NAME, COLLECTION_NAME, MODEL_NAME)

# ========== Simulated User Input ==========
# You can replace this with input() or leave static for testing
user_query = "I want to eat a delicious pizza in a cozy place with vegetarian options"

# ========== Step 1: Parse User Query ==========
parsed_info = recommender.parse_query(user_query)
print("\n🎯 Parsed Info from LLM:")
pprint(parsed_info)

# ========== Step 2: Build MongoDB Query ==========
mongo_query = recommender.build_mongo_query(parsed_info)
print("\n🔍 MongoDB Query:")
pprint(mongo_query)

if isinstance(mongo_query, str):  # If invalid or warning returned
    print(mongo_query)
    exit()

# ========== Step 3: Query Matching Reviews ==========
matching_reviews = recommender.query_reviews(mongo_query)

if not matching_reviews:
    print("⛔ No reviews found for the given query.")
    exit()

# ========== Step 4: Enrich with Review Counts ==========
enriched_reviews = recommender.enrich_results(matching_reviews)
print("\n🔍 Enriched review:")
pprint(enriched_reviews)

# ========== Step 5: Get Top Restaurants ==========
top_restaurants = recommender.get_top_restaurants(enriched_reviews, top_k=4)

print("\n🏆 Top Restaurants:")
pprint(top_restaurants)

# ========== Step 6: Fetch All Full Reviews for Those Restaurants ==========
top_reviews = recommender.fetch_full_reviews(top_restaurants)

print(f"\n📦 Retrieved {len(top_reviews)} full reviews from top restaurants.")
pprint(top_reviews[:3])  # Show just the first 3 for brevity


✅ MongoDB connected.

🎯 Parsed Info from LLM:
{'food_item': ['pizza'], 'preferences': ['cozy', 'vegetarian options']}

🔍 MongoDB Query:
{'$or': [{'aspect_keys': {'$elemMatch': {'$options': 'i', '$regex': 'pizza'}}},
         {'aspect_keys': {'$elemMatch': {'$options': 'i', '$regex': 'cozy'}}},
         {'aspect_keys': {'$elemMatch': {'$options': 'i',
                                         '$regex': 'vegetarian options'}}}]}
[{'Unnamed: 0': 14,
  '_id': ObjectId('68a32681150e99c4db0b6137'),
  'aspect_keys': ['service', 'quality of the pizza', 'staff'],
  'aspects': {'quality of the pizza': 'positive',
              'service': 'positive',
              'staff': 'positive'},
  'author_id': 'UID_14',
  'city': 'Barcelona_Catalonia',
  'date': 'May 17, 2020',
  'parse_count': 15,
  'rating_review': 5,
  'restaurant_name': 'Sports_Bar_Italian_Food_C_Ample',
  'review_count': [3],
  'review_full': 'Despite the lockdown, service at the Sports Bar has been '
                 'outstanding. The